In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval/')
import netCDF4
import fill_values as fv
import datetime
import calc_date as cd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import save_fig as sF
import createFolder as cF
from matplotlib.gridspec import GridSpec
import plt_ce_image as im
import read_MRR as pMRR
import plot_sfc_spaghetti_ret as spagh


from scipy.integrate import simps

In [ ]:
year = '2016'
month = '12'
#t = ['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
t = ['20', '21', '22', '23', '24', '25', '26', '27']
tid = '00'
station = 'Haukeliseter'


### Figures 
savefig = 0
figdir = '../../Figures/Retrieval_MEPS/snow_graupel_cloudice'
cF.createFolder('%s/' %(figdir))
form = 'png'


################################################################
### MEPS
level = 'ml'
nc_dir_ml = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
# snow fall amount
var_name_snow = 'snowfall_amount_ml'
snow_dir = '%s/%s' %(nc_dir_ml,var_name_snow)
# graupel fall amount
var_name_graupel = 'graupelfall_amount_ml'
graupel_dir = '%s/%s' %(nc_dir_ml,var_name_graupel)
# atmosphere_cloud_ice_content_ml
var_name_cl_ice = 'atmosphere_cloud_ice_content_ml'
cl_ice_dir = '%s/%s' %(nc_dir_ml,var_name_cl_ice)

################################################################
### MRR
MRR_dir = '../../Data/MRR/processed_MRR'

################################################################
### Retrieval 
nc_dir_retrieval = '../../Data/Retrieved_SWC'

################################################################

In [ ]:
## MRR ######
fnMRR      = dict()
time_MRR   = dict()
height_MRR = dict()
Ze         = dict()

In [ ]:
## RETRIEVAL ######
fnRet      = dict()
retrieved_snowfall_amount = dict()
SWC        = dict()
SWP_ret    = dict() 

In [ ]:
snow_file         = dict()
graupel_file      = dict()
cl_ice_file       = dict()

fn_snow           = dict()
fn_graupel        = dict()
fn_cl_ice         = dict()

time_ml           = dict()
snow_amount_ml    = dict()
graupel_amount_ml = dict()
cl_ice_ml         = dict()

thickness_ml      = dict()
ice_amount_ml     = dict()
time              = dict()
height_ml         = dict()
height            = dict()
SWP_model         = dict()

In [ ]:
for day in t:
## MRR ######  
    fnMRR[day] = netCDF4.Dataset('%s/VMRR_%s%s%s.nc' %(MRR_dir,year,month,day) ,'r')
    time_MRR[day]   = fnMRR[day].variables['time'][:]
    height_MRR[day] = fnMRR[day].variables['height'][:]

    Ze[day] = pMRR.read_and_mask(fnMRR[day],'Ze', np.nan)         # vertical Ze profile for retrieval

## RETRIEVAL ######
    fnRet[day] = netCDF4.Dataset('%s/out_swc_b6_%s%s%s.nc' %(nc_dir_retrieval, year, month, day))

    retrieved_snowfall_amount[day] = fnRet[day].variables['retrieved_snowfall_amount'][:]
    SWC[day] = fnRet[day].variables['SWC'][:]
    
    swpret = []
    swpret = simps(SWC[day][:,:], height_MRR[day], axis = 0)
    SWP_ret[day] = swpret
# for plotting, only values larger than 0.01
    a,b = np.where(retrieved_snowfall_amount[day][:,:] < 0.01)
    SWC[day][a[:],b[:]] = np.nan
    retrieved_snowfall_amount[day][a[:],b[:]] = np.nan
    np.transpose(Ze[day])[a[:],b[:]] = np.nan

In [ ]:
for k in range(0,np.asarray(t).shape[0]-1):
#for k in range(6,7):

    day = t[k]
    print(day)
## MEPS ######
    for ens_memb in range(0,10):
        snow_file[ens_memb]    = '%s/%s%s%s_%s_%s.nc' %(snow_dir,year,month,day,tid,ens_memb)
        graupel_file[ens_memb] = '%s/%s%s%s_%s_%s.nc' %(graupel_dir,year,month,day,tid,ens_memb)
        cl_ice_file[ens_memb]  = '%s/%s%s%s_%s_%s.nc' %(cl_ice_dir,year,month,day,tid,ens_memb)
    
        fn_snow[ens_memb]      = netCDF4.Dataset(snow_file[ens_memb])
        fn_graupel[ens_memb]   = netCDF4.Dataset(graupel_file[ens_memb]) 
        fn_cl_ice[ens_memb]    = netCDF4.Dataset(cl_ice_file[ens_memb])
# Read in time to know initialisation time (plotting)    
        time_ml[ens_memb] = fn_snow[ens_memb].variables['time']
        time_ml[ens_memb] = fv.fill_nan(time_ml[ens_memb][:])
        

        ini_day = datetime.datetime.utcfromtimestamp(time_ml[0][0]).day   # day of initialisation
        hh = datetime.datetime.utcfromtimestamp(time_ml[0][0]).hour       # first hour of initialisation?
        calday, calmon = cd.get_dayname(year, month, ini_day)
# Read in the variable name which should be plotted  
        snow_amount_ml[ens_memb]    = fn_snow[ens_memb].variables[var_name_snow]
        graupel_amount_ml[ens_memb] = fn_graupel[ens_memb].variables[var_name_graupel]
        cl_ice_ml[ens_memb]         = fn_cl_ice[ens_memb].variables[var_name_cl_ice]


# create an array with time of the shape of the variable
        lead_time   = np.arange(0,snow_amount_ml[ens_memb].shape[0])
        model_level = np.arange(0,snow_amount_ml[ens_memb].shape[1])
        td,Y        = np.meshgrid(lead_time,model_level)

# substitute missing values with nan       
        snow_amount_ml[ens_memb]    = fv.fill_nan(snow_amount_ml[ens_memb][:])
        graupel_amount_ml[ens_memb] = fv.fill_nan(graupel_amount_ml[ens_memb][:])
        cl_ice_ml[ens_memb]         = fv.fill_nan(cl_ice_ml[ens_memb][:])
# get ice amount (snow+graupel)
        ice_amount_ml[ens_memb]     = snow_amount_ml[ens_memb] + graupel_amount_ml[ens_memb] + cl_ice_ml[ens_memb]
    
# layer thickness to calculate height
        thickness_ml[ens_memb]      = fn_snow[ens_memb].variables['layer_thickness']
        thickness_ml[ens_memb]      = fv.fill_nan(thickness_ml[ens_memb][:])
    
        thickness = thickness_ml[ens_memb]
        h_above = []
        h_above.append(np.zeros(thickness[:,:].shape[0]))
        th_arr = np.nansum([np.asarray(h_above)[:,0], thickness[:,0]], axis = 0)
        h_above.append(th_arr)
        for i in range(2,thickness[:,:].shape[1]):
            th_arr = np.nansum([h_above[i-1], thickness[:,i-1]], axis = 0)
            h_above.append(th_arr)
        height_ml[ens_memb] = np.transpose(h_above)
        
        
# get only valuable values not nan    
        time_ml[ens_memb]       = (time_ml[ens_memb][~np.isnan(time_ml[ens_memb])])
        time[ens_memb]          = (td[:,~np.isnan(ice_amount_ml[ens_memb]).any(axis=1)])
        height[ens_memb]        = (height_ml[ens_memb][~np.isnan(ice_amount_ml[ens_memb]).any(axis=1),:])
        thickness_ml[ens_memb]  = (thickness_ml[ens_memb][~np.isnan(ice_amount_ml[ens_memb]).any(axis=1),:])  
        ice_amount_ml[ens_memb] = (ice_amount_ml[ens_memb][~np.isnan(ice_amount_ml[ens_memb]).any(axis=1)])

# convert the snow amount from [kg/m^2] into [g/m^2] by multiply with 1000
        ice_amount_ml[ens_memb] = (ice_amount_ml[ens_memb]*1000.)

# calculate SWP from model levels
        SWP_ml = []
        for i in range(0,ice_amount_ml[ens_memb][:,:].shape[0]):
            SWP_ml.append(simps(y = ice_amount_ml[ens_memb][i,np.where(height[ens_memb][i,:] <= 3000)], 
                            x = height[ens_memb][i,np.where(height[ens_memb][i,:] < 3000)]))
        SWP_model[ens_memb] = SWP_ml

        h1, h2 = np.where(height[ens_memb][:,:] > 3000)
        ice_amount_ml[ens_memb][h1[:],h2[:]] = np.nan
        time[ens_memb][h2[:],h1[:]]          = -9999
        height[ens_memb][h1[:],h2[:]]        = np.nan
    

###################################################################################
    if Ze[day].shape[0] > 1440:
   #     print('Ze ==', Ze[day].shape[0])
        df1 = Ze[day][:-1]
        df2 = Ze[str(int(day)+1)][1::,:]
    elif Ze[str(int(day)+1)].shape[0] > 1440:
  #      print('Ze(day+1) == ', Ze[str(int(day)+1)].shape[0])
        df1 = Ze[day]
        df2 = Ze[str(int(day)+1)][1:-1,:]
    else:
        df1 = Ze[day]
        df2 = Ze[str(int(day)+1)][1::,:]
    mergedZe = []
    mergedZe.extend(df1)
    mergedZe.extend(df2)

    np.asarray(mergedZe).shape

    mergedSWC = []
    mergedSWC.extend(np.transpose(SWC[day]))
    mergedSWC.extend(np.transpose(SWC[str(int(day)+1)][:,1::][:]))

    np.asarray(mergedSWC).shape

    mergedSWP = []
    mergedSWP.extend(SWP_ret[day])
    mergedSWP.extend(SWP_ret[str(int(day)+1)][1::][:])

    np.asarray(mergedSWP).max()
    np.asarray(mergedSWP).shape

###########################################################################
### PLOTTING ###########

    xdays = ['%s-%s-%s' %(year,month,ini_day), 
        6,12, 18,
        '%s-%s-%s' %(year,month,ini_day+1), 
        6, 12, 18,
        '%s-%s-%s' %(year,month,ini_day+2)]
    
    
    fig_name = '%s%s%s_48.%s' %(year,month,ini_day,form)
    fig = plt.figure(figsize=(20.,25))

    gs = GridSpec(4,2)
## 1st subplot --- REFLECTIVITY
    ax0 = plt.subplot(gs[0,:])
# Vertical line to show end of day 
    ax0.axvline(0,color = spagh.vert_col, linewidth = 3)
    ax0.axvline(time_MRR[day].max(),color = spagh.vert_col, linewidth = 3)
    ax0.axvline(2*time_MRR[day].max(),color = spagh.vert_col, linewidth = 3)
    levels = np.arange(-10,30.1,0.1)
    if day == '26':
        im.plt_ce_image(fig,ax0,np.arange(0,2*time_MRR[day].max()-60,60), height_MRR[day], np.transpose(mergedZe),levels,
            v_min=0.0, v_max=30., xmax=2*time_MRR[day].max(), ymax=height_MRR[day].max(), 
            xticks=np.arange(0,60*60*49,6*60*60), xlabel= xdays, yticks=im.yticks1,
            cb_ticks=levels[::50],var_label='Ze [dBZ]',cb=1) 
    else:
        im.plt_ce_image(fig,ax0,np.arange(0,2*time_MRR[day].max()+1,60), height_MRR[day], np.transpose(mergedZe),levels,
            v_min=0.0, v_max=30., xmax=2*time_MRR[day].max(), ymax=height_MRR[day].max(), 
            xticks=np.arange(0,60*60*49,6*60*60), xlabel= xdays, yticks=im.yticks1,
            cb_ticks=levels[::50],var_label='Ze [dBZ]',cb=1)
    ax0.xaxis.grid()
#    ax0.text(0.02,0.95, '%s, %s %s %s' %(calday, day, calmon, year),
 #           verticalalignment ='top', horizontalalignment='left',
  #           transform = ax0.transAxes,
   #          color = im.date_blue, fontsize = im.fontsize,
    #         bbox={'facecolor':'white','alpha':1.,'pad':10})

################
## 2nd subplot --- SNOW WATER CONTENT RETRIEVAL
    ax1 = plt.subplot(gs[1,:])
# Vertical line to show end of day 
    ax1.axvline(0,color = spagh.vert_col, linewidth = 3)
    ax1.axvline(time_MRR[day].max(),color = spagh.vert_col, linewidth = 3)
    ax1.axvline(2*time_MRR[day].max(),color = spagh.vert_col, linewidth = 3)
    levels = np.arange(0,1.51,0.01) 
    if day == '26':
        im.plt_ce_image(fig,ax1,np.arange(0,2*time_MRR[day].max()-60,60),height_MRR[day], np.transpose(mergedSWC),levels, 
             v_min=0., v_max=1.5,xmax=2*time_MRR[day].max(), ymax=height_MRR[day].max(), 
             xticks=np.arange(0,60*60*49,6*60*60), xlabel= xdays, yticks=im.yticks1,
             cb_ticks=levels[::30], var_label='SWC [g m$^{-3}$]',cb=1)
    else:
        im.plt_ce_image(fig,ax1,np.arange(0,2*time_MRR[day].max()+1,60),height_MRR[day], np.transpose(mergedSWC),levels, 
             v_min=0., v_max=1.5,xmax=2*time_MRR[day].max(), ymax=height_MRR[day].max(), 
             xticks=np.arange(0,60*60*49,6*60*60), xlabel= xdays, yticks=im.yticks1,
             cb_ticks=levels[::30], var_label='SWC [g m$^{-3}$]',cb=1)
    ax1.xaxis.grid()

################
## 3rd subplot --- SNOW WATER CONTENT ML
    ax2 = plt.subplot(gs[2,:])
# Vertical line to show end of day 
    ax2.axvline(0,color = spagh.vert_col, linewidth = 3)
    ax2.axvline(24,color = spagh.vert_col, linewidth = 3)
    ax2.axvline(48,color = spagh.vert_col, linewidth = 3)
    levels = np.arange(0,1.51,0.01) 
    im.plt_ce_image(fig,ax2,np.transpose(time[0]), height[0], (ice_amount_ml[0]),levels, 
            v_min=0., v_max=1.5, xmax=48., ymax=3000., 
            xticks =np.arange(0,49,6), xlabel= xdays, yticks=im.yticks1,
            cb_ticks=levels[::30], var_label='SWC$_{MEPS}$(EM0) [g m$^{-2}$]',cb=1)
    ax2.xaxis.grid()
################
## 4th subplot
    ax4 = plt.subplot(gs[3,:])
# Vertical line to show end of day
    ax4.axvline(0,color = spagh.vert_col, linewidth = 3)
    ax4.axvline(24,color = spagh.vert_col, linewidth = 3)
    ax4.axvline(48,color = spagh.vert_col, linewidth = 3)
# ensemble members
    for ens_memb in range(2,10):
        ax4.plot(time[ens_memb][0,:], SWP_model[ens_memb], color = im.memb_col, linestyle='-', label='_nolegend_')
    ax4.plot(time[1][0,:], SWP_model[1], color = im.memb_col, linestyle='-', label='ensemble member [g m$^{-1}$]')
    ax4.plot(time[0][0,:], SWP_model[0], 'k', linewidth=4, label='best guess [g m$^{-1}$]')
    
    if day == '26':
        line_ret = ax4.plot(np.arange(0,2*time_MRR[day].max()-60,60)/(60*60), 
                            mergedSWP, label = 'retrieval [g m$^{-2}$]')
    else:
        line_ret = ax4.plot(np.arange(0,2*time_MRR[day].max()+60,60)/(60*60), 
                    mergedSWP, label = 'retrieval [g m$^{-2}$]')
    ax4.axis([0, 48, 0, 6000.])

### fine tuning
    lgd = plt.legend(loc='upper left',fontsize=im.label_fs)
    plt.grid()
# labels

    im.labels_x(ax4,np.arange(0,49,6),xdays)
    im.labels_y(ax4,im.yticks2,im.yl2, 'SWP')



################
    plt.tight_layout(pad=1.4, w_pad=1.5, h_pad=1.5)

    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
        print('saved: %s/%s' %(figdir, fig_name))
    else:
        plt.show()
    plt.close()
    
for ens_memb in range(0,10):
        fn_snow[ens_memb].close()
        fn_graupel[ens_memb].close()